# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [1]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 2.5 MB/s eta 0:00:00


In [2]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [3]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [5]:
states_A = set(range(3))
actions_A = set(range(1))

print(f'{states_A=} | {actions_A=}')

T_A = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}
print(f'A transition function {T_A=}')

A_mdp = states_A, actions_A, T_A

states_B = set(range(4))
actions_B = set(range(3))

print(f'{states_B=} | {actions_B=}')

T_B = {
    (0, 0): 1,
    (0, 1): 2,
    (0, 2): 3,
    (1, 0): 1,
    (1, 1): 1,
    (1, 2): 1,
    (2, 0): 2,
    (2, 1): 2,
    (2, 2): 2,
    (3, 0): 3,
    (3, 1): 3,
    (3, 2): 3
}

print(f'B transition function {T_B=}')
B_mdp = states_B, actions_B, T_B

states_A={0, 1, 2} | actions_A={0}
A transition function T_A={(0, 0): 1, (1, 0): 2, (2, 0): 2}
states_B={0, 1, 2, 3} | actions_B={0, 1, 2}
B transition function T_B={(0, 0): 1, (0, 1): 2, (0, 2): 3, (1, 0): 1, (1, 1): 1, (1, 2): 1, (2, 0): 2, (2, 1): 2, (2, 2): 2, (3, 0): 3, (3, 1): 3, (3, 2): 3}


Попробуйте записать функцию перехода в матричном виде:

In [7]:
T_A_np = np.array([
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 1]
])

T_B_np = np.array([
    [0, 1, 1, 1],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [18]:
n = 2
b = np.linalg.matrix_power(T_A_np, n)

print(b[0, 1])

n = 2000
b = np.linalg.matrix_power(T_A_np, n)

print(b[0, 1])

0
0


Задайте еще несколько MDP:

- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

In [19]:
states_C = set(range(4))
actions_C = set(range(2))

print(f'{states_C=} | {actions_C=}')

T_C = {
    (0, 0): 1,
    (0, 1): 2,

    (1, 0): 1,
    (1, 1): 3,

    (2, 0): 3,
    (2, 1): 2,

    (3, 0): 3,
    (3, 1): 3
}
print(f'C transition function {T_C=}')
C_mdp = states_C, actions_C, T_C

T_C_np = np.array([
    [0, 1, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 1],
    [0, 0, 0, 1]
])

n = 2
b = np.linalg.matrix_power(T_C_np, n)

print(b[0, 1])

n = 2000
b = np.linalg.matrix_power(T_C_np, n)

print(b[0, 1])

states_C={0, 1, 2, 3} | actions_C={0, 1}
C transition function T_C={(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 3, (2, 1): 2, (3, 0): 3, (3, 1): 3}
1
1


Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [23]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))

В качестве дополнения, запишите стратегию агента

In [24]:
class OptimalAgent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return 0

## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [21]:
R = {
    (0, 0): -0.1,
    (1, 0): 1.0,
    (2, 0): 0.0
}
print(R)

A_mdp = *A_mdp, R
print(A_mdp)

{(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}
({0, 1, 2}, {0}, {(0, 0): 1, (1, 0): 2, (2, 0): 2}, {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0})


## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [25]:
def run_episode(mdp):
    states, actions, T, R = mdp
    agent = Agent(actions)

    s = 0
    tau = []
    for _ in range(5):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

    return tau

run_episode(A_mdp)

[(0, np.int64(0), -0.1),
 (1, np.int64(0), 1.0),
 (2, np.int64(0), 0.0),
 (2, np.int64(0), 0.0),
 (2, np.int64(0), 0.0)]

Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [26]:
def G_mean(mdp, N, gamma, t=0):
  sum_total = 0
  for i in range(N):
    T = run_episode(mdp)
    sum_T = 0
    for step_i in range(t, len(T)):
        s, a, r = T[step_i]
        sum_T += r * gamma**(step_i-t)
    sum_total += sum_T
  return sum / N